In [1]:
import sys
import os
import cv2
import json
import yaml
import numpy as np
import torch
import glob
import shutil
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# change this property
NOMEROFF_NET_DIR = os.path.abspath('../../../../')
sys.path.append(NOMEROFF_NET_DIR)

In [2]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ['CUDA_VISIBLE_DEVICES']='1'

In [3]:
plt.rcParams["figure.figsize"] = (20, 10)

In [4]:
%matplotlib inline 

In [5]:
PATH_TO_DATASET = os.path.join(NOMEROFF_NET_DIR, "./data/dataset/FraudDetector/fraud_numberplate/generated")

In [6]:
from nomeroff_net.pipes.number_plate_classificators.fraud_detector import FraudDetector

In [7]:
# definde your parameters
class MyNpClassificator(FraudDetector):
    def __init__(self):
        FraudDetector.__init__(self)
        
        self.epochs = 100
        self.batch_size = 128
        
        # count gpu for train
        self.gpus = torch.cuda.device_count()
        
        self.height = 256
        self.width = 256


In [8]:
# initialize region detector.
npClassificator = MyNpClassificator()
npClassificator.prepare(PATH_TO_DATASET, verbose=1, num_workers=1)

START PREPARING


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4456/4456 [00:00<00:00, 70276.21it/s]

DATA PREPARED


In [9]:
npClassificator

In [10]:
PATH_TO_DATASET

'/mnt/raid/var/www/nomeroff-net/./data/dataset/FraudDetector/fraud_numberplate/generated'

In [11]:
# # train
# lr_finder = npClassificator.tune()

# # Plot with
# fig = lr_finder.plot(suggest=True)
# fig.show()

In [ ]:
# train
model = npClassificator.train()

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [1]

  | Name       | Type              | Params
-------------------------------------------------
0 | criterion  | BCEWithLogitsLoss | 0     
1 | resnet     | Sequential        | 2.8 M 
2 | fc1        | Linear            | 33.6 M
3 | fc2        | Linear            | 131 K 
4 | fc3        | Linear            | 257   
5 | relu       | ReLU              | 0     
6 | dropout    | Dropout           | 0     
7 | batchnorm1 | BatchNorm1d       | 1.0 K 
8 | batchnorm2 | BatchNorm1d       | 512   
-------------------------------------------------
36.5 M    Trainable params
0         Non-trainable params
36.5 M    Total params
145.883   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

/usr/local/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 32 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/usr/local/lib64/python3.9/site-packages/torch/nn/functional.py:1806: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/usr/local/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 32 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

/usr/local/lib/python3.9/site-packages/pytorch_lightning/loops/optimization/closure.py:35: LightningDeprecationWarning: One of the returned values {'log', 'progress_bar'} has a `grad_fn`. We will detach it automatically but this behaviour will change in v1.6. Please detach it manually: `return {'loss': ..., 'something': something.detach()}`
  rank_zero_deprecation(


In [ ]:
npClassificator.save("fake")

In [ ]:
imgs = [cv2.resize(mpimg.imread(img_path)[:,:,:3], (256, 256))
        for img_path in glob.glob(os.path.join(NOMEROFF_NET_DIR, "./data/examples/fake_np_grabed/*"))]

In [ ]:
for img in imgs:
    plt.imshow(img)
    plt.show()

In [ ]:
model.forward(torch.from_numpy(np.moveaxis(np.array(imgs), 3, 1)))